In [63]:
import pandas as pd 
order=pd.read_csv("orders.csv")
print(order.head())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  


In [65]:
user=pd.read_json("users.json")
print(user.head())

   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold


In [64]:
import sqlite3

conn = sqlite3.connect("restaurat.db")
cursor = conn.cursor()

# Read SQL file
with open("restaurants.sql", "r") as f:
    sql_script = f.read()

# Execute the script
cursor.executescript(sql_script)
conn.commit()
conn.close()


OperationalError: table restaurants already exists

In [18]:
conn = sqlite3.connect("restaurat.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall()) 
conn.close()


[('restaurants',)]


In [66]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("restaurants.db")

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in DB:", tables)
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
print(restaurants.head())

conn.close()


Tables in DB:           name
0  restaurants
   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [67]:
orders_users=pd.merge(order,user,on='user_id',how='left')
final_dataset=pd.merge(orders_users,restaurants,on='restaurant_id',how='left')
print(final_dataset.head())
print(final_dataset.info())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3    Restaurant_224  Chinese    

In [68]:
final_dataset.to_csv("final_food_delivery_dataset.csv")

In [69]:
final=pd.read_csv("final_food_delivery_dataset.csv")
print(final)

      Unnamed: 0  order_id  user_id  restaurant_id  order_date  total_amount  \
0              0         1     2508            450  18-02-2023        842.97   
1              1         2     2693            309  18-01-2023        546.68   
2              2         3     2084            107  15-07-2023        163.93   
3              3         4      319            224  04-10-2023       1155.97   
4              4         5     1064            293  25-12-2023       1321.91   
...          ...       ...      ...            ...         ...           ...   
9995        9995      9996     2528            249  21-05-2023       1211.96   
9996        9996      9997     2867            267  06-08-2023       1188.05   
9997        9997      9998      522            420  11-11-2023        979.44   
9998        9998      9999      319            492  08-09-2023       1105.93   
9999        9999     10000      457            439  21-10-2023        879.58   

                   restaurant_name_x   

In [84]:
import pandas as pd
import matplotlib
matplotlib.use("Agg")  
import matplotlib.pyplot as plt

df = pd.read_csv("final_food_delivery_dataset.csv")
# Remove Unnamed column
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])
# Convert dates
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)
# Extract useful time fields 
df['Month'] = df['order_date'].dt.to_period('M') 
df['Day'] = df['order_date'].dt.day_name()

print("\nCOLUMNS FOUND IN DATASET:\n", df.columns)



COLUMNS FOUND IN DATASET:
 Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'Month', 'Day'],
      dtype='object')


In [85]:
# ORDER TRENDS
monthly_orders = df.groupby('Month')['order_id'].count()
monthly_revenue = df.groupby('Month')['total_amount'].sum()

print("\nMonthly Orders:\n", monthly_orders)
print("\nMonthly Revenue:\n", monthly_revenue)

plt.figure(figsize=(10,5))
monthly_orders.plot(kind="line")
plt.title("Orders Over Time")
plt.savefig("orders_over_time.png")
plt.close()


Monthly Orders:
 Month
2023-01    804
2023-02    785
2023-03    903
2023-04    812
2023-05    844
2023-06    784
2023-07    859
2023-08    851
2023-09    812
2023-10    863
2023-11    807
2023-12    849
2024-01     27
Freq: M, Name: order_id, dtype: int64

Monthly Revenue:
 Month
2023-01    645818.76
2023-02    630867.40
2023-03    716738.98
2023-04    666097.18
2023-05    668428.61
2023-06    610822.93
2023-07    688559.45
2023-08    694987.58
2023-09    653838.07
2023-10    699187.73
2023-11    637772.43
2023-12    681303.50
2024-01     17201.50
Freq: M, Name: total_amount, dtype: float64


In [72]:
# USER ANALYSIS
user_spending = df.groupby('user_id')['total_amount'].sum()
print("\nTop 10 spending users:\n", user_spending.sort_values(ascending=False).head(10))


Top 10 spending users:
 user_id
1515    11556.49
650     10747.44
496      9634.30
2586     9486.61
2615     9237.32
636      9087.09
2973     9063.07
874      8916.55
1337     8907.50
2694     8875.71
Name: total_amount, dtype: float64


In [73]:
# CITY ANALYSIS (ONLY IF CITY EXISTS)
if "city" in df.columns:
    city_orders = df.groupby("city")["order_id"].count()
    city_revenue = df.groupby("city")["total_amount"].sum()

    print("\nCity-wise Orders:\n", city_orders)
    print("\nCity-wise Revenue:\n", city_revenue)
else:
    print("\n WARNING: No 'city' column found. Skipping city-wise analysis.")


City-wise Orders:
 city
Bangalore    2751
Chennai      2469
Hyderabad    2350
Pune         2430
Name: order_id, dtype: int64

City-wise Revenue:
 city
Bangalore    2206946.58
Chennai      1990513.03
Hyderabad    1889366.58
Pune         1924797.93
Name: total_amount, dtype: float64


In [53]:
# CUISINE ANALYSIS

if "cuisine" in df.columns:
    cuisine_orders = df.groupby("cuisine")["order_id"].count()
    cuisine_revenue = df.groupby("cuisine")["total_amount"].sum()
    cuisine_restaurants = df.groupby("cuisine")["restaurant_id"].nunique()

    print("\nCuisine-wise Orders:\n", cuisine_orders)
    print("\nCuisine-wise Revenue:\n", cuisine_revenue)
    print("\nDistinct Restaurants per Cuisine:\n", cuisine_restaurants)
else:
    print("\n WARNING: No 'cuisine' column found. Skipping cuisine analysis.")


Cuisine-wise Orders:
 cuisine
Chinese    2418
Indian     2469
Italian    2532
Mexican    2581
Name: order_id, dtype: int64

Cuisine-wise Revenue:
 cuisine
Chinese    1930504.65
Indian     1971412.58
Italian    2024203.80
Mexican    2085503.09
Name: total_amount, dtype: float64

Distinct Restaurants per Cuisine:
 cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64


In [74]:
#MEMBERSHIP ANALYSIS
if "membership" in df.columns:
    membership_orders = df.groupby("membership")["order_id"].count()
    membership_revenue = df.groupby("membership")["total_amount"].sum()

    membership_share = (membership_orders / membership_orders.sum()) * 100

    print("\nOrders by Membership:\n", membership_orders)
    print("\nRevenue by Membership:\n", membership_revenue)
    print("\nMembership % Share:\n", membership_share.round(2))
else:
    print("\n WARNING: No 'membership' column found. Skipping membership analysis.")


Orders by Membership:
 membership
Gold       4987
Regular    5013
Name: order_id, dtype: int64

Revenue by Membership:
 membership
Gold       3975364.89
Regular    4036259.23
Name: total_amount, dtype: float64

Membership % Share:
 membership
Gold       49.87
Regular    50.13
Name: order_id, dtype: float64


In [55]:
# REVENUE DISTRIBUTION
plt.figure(figsize=(8,5))
df["total_amount"].plot(kind="hist", bins=30)
plt.title("Revenue Distribution")
plt.savefig("revenue_distribution.png")
plt.close()

print("\nEDA Completed Successfully.")


EDA Completed Successfully.


In [75]:
print(df.columns)

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'Month', 'Day'],
      dtype='object')


In [86]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

# Filter only Gold members
gold_df = df[df["membership"] == "Gold"]

# Group by city and sum revenue
city_revenue = gold_df.groupby("city")["total_amount"].sum()

print(city_revenue)
print("\nHighest Revenue City:", city_revenue.idxmax())


city
Bangalore     994702.59
Chennai      1080909.79
Hyderabad     896740.19
Pune         1003012.32
Name: total_amount, dtype: float64

Highest Revenue City: Chennai


In [95]:
avg_order_value = df.groupby("cuisine")["total_amount"].mean()
print("\nAverage Order Value by Cuisine:\n", avg_order_value)
print("\nHighest Average Order Value:", avg_order_value.idxmax())


Average Order Value by Cuisine:
 cuisine
Chinese    798.389020
Indian     798.466011
Italian    799.448578
Mexican    808.021344
Name: total_amount, dtype: float64

Highest Average Order Value: Mexican


In [91]:
user_total = df.groupby("user_id")["total_amount"].sum()
high_spenders = user_total[user_total > 1000]
count_users = len(high_spenders)
print("Number of users who spent > 1000:", count_users)

Number of users who spent > 1000: 2544


In [92]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)

# Total revenue by rating range
revenue = df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)

print(revenue)
print("\nHighest Revenue Rating Range:", revenue.idxmax())

rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64

Highest Revenue Rating Range: 4.6 – 5.0


In [94]:
gold_df=df[df["membership"]=="Gold"]
avg_city=gold_df.groupby("city")["total_amount"].mean()
print(avg_city)
print("\n Highest Avg Order value city:",avg_city.idxmax())

city
Bangalore    793.223756
Chennai      808.459080
Hyderabad    806.421034
Pune         781.162243
Name: total_amount, dtype: float64

 Highest Avg Order value city: Chennai


In [101]:
distinct_rest = df.groupby("cuisine")["restaurant_id"].nunique()
revenue = df.groupby("cuisine")["total_amount"].sum()
summary = pd.DataFrame({
    "distinct_restaurants": distinct_rest,
    "total_revenue": revenue
})
print(summary.sort_values("distinct_restaurants"))

         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65
Indian                    126     1971412.58
Italian                   126     2024203.80
Mexican                   128     2085503.09


In [102]:
gold_orders=df[df["membership"]=="Gold"].shape[0]
total_orders=df.shape[0]
percent=round((gold_orders/total_orders)*100)
print("percent:",percent,"%")

percent: 50 %


In [142]:
restaurant_stats = df.groupby('restaurant_name_x').agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
)
few_orders = restaurant_stats[restaurant_stats['total_orders'] < 20]
few_orders['avg_order_value'].idxmax()


'Hotel Dhaba Multicuisine'

In [146]:
filtered_df = final[
    ((final['membership'] == 'Gold') & (final['cuisine'] == 'Indian')) |
    ((final['membership'] == 'Gold') & (final['cuisine'] == 'Italian')) |
    ((final['membership'] == 'Regular') & (final['cuisine'] == 'Indian')) |
    ((final['membership'] == 'Regular') & (final['cuisine'] == 'Chinese'))
]
option_revenue = filtered_df.groupby(
    ['membership', 'cuisine']
)['total_amount'].sum()
option_revenue.idxmax()


('Gold', 'Italian')

In [148]:

final['order_date'] = pd.to_datetime(final['order_date'], dayfirst=True)
final['quarter'] = final['order_date'].dt.quarter
revenue_by_quarter = final.groupby('quarter')['total_amount'].sum()
revenue_by_quarter.idxmax()


3

In [132]:
#What is the average order value (rounded to 2 decimals) for Gold members?
avg_order_val_gold=final[final['membership']=='Gold']['total_amount'].mean()
round(avg_order_val_gold,2)

797.15

In [133]:
#What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
hyd_rev=final[final['city']=='Hyderabad']['total_amount'].sum()
round(hyd_rev)

1889367

In [135]:
#How many total orders were placed by users with Gold membership?
gold_ord_cnt=final[final['membership']=='Gold'].shape[0]
gold_ord_cnt

4987

In [131]:
#How many distinct users placed at least one order?
distinct_users=final['user_id'].nunique()
distinct_users

2883

In [123]:
#How many orders were placed for restaurants with rating ≥ 4.5?
high_rating_orders = final[final['rating'] >= 4.5]
high_rating_orders_count = high_rating_orders.shape[0]
high_rating_orders_count


3374

In [121]:
#How many orders were placed in the top revenue city among Gold members only?
gold_chennai_orders = final[
    (final['membership'] == 'Gold') &
    (final['city'] == 'Chennai')
]

gold_chennai_orders_count = gold_chennai_orders.shape[0]

gold_chennai_orders_count


1337